<a href="https://colab.research.google.com/github/yigalk89/hatespeech-adversarial-attacks/blob/master/Create_perturbed_anthro_data_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hate explain dataset perturbation using ANTHRO

### ANTHRO set up

In [ ]:
# Setup Anthro (word replacement tool)
!git clone https://github.com/lethaiq/perturbations-in-the-wild.git
!pip install Levenshtein
!pip install homoglyphs
!pip install pyspellchecker

import numpy as np
import pandas as pd
import glob
import json
import pickle
import re
import Levenshtein
from spacy.tokenizer import Tokenizer
from tqdm import tqdm
import os
#import enchant
from spellchecker import SpellChecker
from nltk.stem import PorterStemmer
import homoglyphs as hg
from spacy.lang.en import English


#english_checker = enchant.Dict("en_US")
english_checker = SpellChecker()
ps = PorterStemmer()
homoglyphs = hg.Homoglyphs(languages={'en'}, strategy=hg.STRATEGY_LOAD)

'''
def is_english(token):
    if 'our' in token.lower() and english_checker.check(token.lower().replace('our','or')):
        return True
    if 'able' in token.lower() and english_checker.check(token.lower().replace('able','ible')):
        return True
    if 'ise' in token.lower() and english_checker.check(token.lower().replace('ise','ize')):
        return True
    if english_checker.check(token.lower()) or english_checker.check(ps.stem(token)):
        return True
    return False
'''
def is_english(token):
  token_lower = token.lower()
  if 'our' in token_lower and english_checker.correction(token_lower.replace('our', 'or')) == token_lower.replace('our', 'or'):
    return True
  if 'able' in token_lower and english_checker.correction(token_lower.replace('able', 'ible')) == token_lower.replace('able', 'ible'):
    return True
  if 'ise' in token_lower and english_checker.correction(token_lower.replace('ise', 'ize')) == token_lower.replace('ise', 'ize'):
      return True
  if english_checker.correction(token_lower) == token_lower:
    return True
  return False

def matchcodex(token, pre_suf_chars=3):
    token = re.sub('[1]+', 'l', token, flags=re.I)
    token = re.sub('[^0-9a-zA-Z]+', '', token, flags=re.I)
    token = token.replace('*','').replace('+','t').replace('!','i')
    token = token.upper()
    codex = []
    prev = None
    for i, c in enumerate(token):
        if c != " " and (i == 0 and c in "A@EI!O0UY") or (c not in "A@EI!O0UY" and c != prev):
            codex.append(c)
        prev = c
    if len(codex) > int(pre_suf_chars*2):
        return "".join(codex[:pre_suf_chars] + codex[-pre_suf_chars:])
    else:
        return "".join(codex)


def preprocess_onebyone(texts, batch_size=100):
    tokenizer = Tokenizer(English().vocab, token_match=re.compile(r'\S+').match)
    preproc_pipe = []
    for doc in tokenizer.pipe(texts, batch_size=batch_size):
        preproc_pipe.append(doc)
    return preproc_pipe

def good_pair_replacement(a, b):
    if a == b:
        return True
    a = a.replace('!','i').replace('+','t')
    b = b.replace('!','i').replace('+','t')

    pairs = {'iy', 'yi', '@a','a@','0o','o0','5s','s5','1l','l1','6b','b6','9g','g9','t+','+t'}
    if "{}{}".format(a,b) in pairs:
        return True
    return False


def good_pair_nearby(a, b):
    if a == b:
        return True
    pairs = {'iy', 'yi', 'ie', 'ye', '@a','a@','0o','o0', 'ya', 'uy', 'ue', 'ae'}
    if "{}{}".format(a,b) in pairs:
        return True
    return False


def check_vowel(a):
    VOWELS = 'a@eijo0uy!'
    try:
        a = homoglyphs.to_ascii(a)[0]
    except:
        return False # change this part that applies to both mfalse and mtrue
    if a in VOWELS:
        return True
    return False


def final_check(token, match):
    if is_english(match.lower()) and match.lower() != token.lower() and not is_english(token.lower()):
        return True

    token = token.lower()
    match = match.lower()
    edits = Levenshtein.editops(token, match)

    for edit in edits:
        if edit[0] != 'delete':
            try:
                c = match[edit[2]]

                if edit[0] == 'replace':
                    try:
                        b_c = token[edit[1]-1]
                    except:
                        b_c = None

                    try:
                        a_c = token[edit[1]+1]
                    except:
                        a_c = None

                    o_c = token[edit[1]]

                    if check_vowel(c):
                        if not good_pair_replacement(o_c, c):
                            return True

                        if not check_vowel(o_c):
                            return True

                    else:
                        if c.isdigit() and not good_pair_replacement(o_c, c):
                            return True
                        if check_vowel(b_c) and check_vowel(a_c):
                            return True

                if edit[0] == 'insert' and check_vowel(c):
                    try:
                        b_c = token[edit[1]-1]
                    except:
                        b_c = None

                    try:
                        a_c = token[edit[1]]
                    except:
                        a_c = None

                    if b_c != None and not good_pair_nearby(b_c, c) and check_vowel(b_c):
                        return True

                    if a_c != None and not good_pair_nearby(c, a_c) and check_vowel(a_c):
                        return True

                    if b_c != None and a_c != None and check_vowel(b_c) and check_vowel(a_c) and c != b_c and c != a_c:
                        return True

                    if b_c != None and a_c != None and not check_vowel(b_c) and not check_vowel(a_c):
                        return True

            except Exception as e:
                print(e)
                print("ERROR!!! => EDIT", edit, token, match)

    return False


class ANTHRO:
    def __init__(self):
        self.max_level = 2
        self.token2idx = {}
        self.idx2token = {}
        self.level2code2token = {}
        self.token2spell = {}
        self.__version__ = 'Public'
        self.preserve_last_threshold = 3

        for level in range(self.max_level):
            self.level2code2token[str(level)] = {}


    def save(self, path, data=None):
        from json import JSONEncoder
        class set_encoder(JSONEncoder):
            def default(self, obj):
                return list(obj)

        if not data:
            data = {
                'token2idx': self.token2idx,
                'level2code2token': self.level2code2token,
                'token2spell': self.token2spell,
            }

        def set_default(obj):
            if isinstance(obj, set):
                return list(obj)
            raise TypeError

        try:
            os.makedirs("{}/".format(path))
        except:
            pass

        for k in data:
            print("saving...", k)
            with open("{}/{}.json".format(path, k), 'w', encoding='utf-8') as f:
                json.dump(data[k].items(), f, cls=set_encoder)

    def load(self, path):
        keys = {
            'token2idx': 'token2idx',
            'level2code2token': 'level2code2token',
            'token2spell': 'token2spell'
        }

        data = {}
        for k in keys:
            print("loading...", k)
            with open("{}/{}.json".format(path, keys[k]), 'r', encoding='utf-8') as f:
                data[k] = json.load(f)
                setattr(self, k, dict(data[k]))

        print("loaded")
        self.statistics()

        for level in self.level2code2token:
            codes = self.level2code2token[str(level)]
            for code in codes:
                codes[code] = set(codes[code])

        self.convert_token_idx()
        self.fix_token2spell()

        print("done")


    def fix_token2spell(self):
        for token in tqdm(self.token2spell.keys()):
            if 'list' in str(type(self.token2spell[token])) and len(self.token2spell[token]) > 0:
                if 'list' in str(type(self.token2spell[token][0])):
                        self.token2spell[token] = set(list(np.concatenate(self.token2spell[token])))


    def add_raw_tokens(self, raw_tokens, level):
        codes = self.level2code2token[str(level)]
        for token in tqdm(raw_tokens):
            token = str(token)
            token_lower = token.lower()

            if token not in self.token2idx:
                self.token2idx[token] = len(self.token2idx)
            if token_lower not in self.token2idx:
                self.token2idx[token_lower] = len(self.token2idx)
            code = self.soundex(token_lower, k=level)

            if code not in codes:
                codes[code] = set()
            if self.token2idx[token_lower] not in codes[code]:
                codes[code].add(self.token2idx[token_lower])

            if self.token2idx[token_lower] not in self.token2spell:
                self.token2spell[self.token2idx[token_lower]] = set()
            self.token2spell[self.token2idx[token_lower]].add(self.token2idx[token])

    def process(self, texts, raw_tokens=None):
        if not raw_tokens:
            sentence_tokens = preprocess_onebyone(texts)

            raw_tokens = []
            for j, tokens in tqdm(enumerate(sentence_tokens)):
                tokens = list(tokens)
                raw_tokens.extend(tokens)

        for level in range(self.max_level):
            if level > 0:
                print("...level", level)
                self.add_raw_tokens(raw_tokens, level)

        self.convert_token_idx()

    def convert_token_idx(self):
        for token in tqdm(self.token2idx):
            self.idx2token[int(self.token2idx[token])] = token

    def statistics(self):
        print("UPPD VERSION", self.__version__)
        print("Total unique codes:")
        for level in self.level2code2token:
            codes = self.level2code2token[str(level)]
            print("Level {}: {} codes".format(level, len(codes)))

    def soundex(self, token, k=0):
        soundex_result = token[:k+1].upper()
        token = re.sub('[hw]', '', token, flags=re.I)
        token = re.sub('[-]','', token, flags=re.I)
        token = re.sub('[l1]+', '4', token, flags=re.I)
        token = re.sub('[bfpv]+', '1', token, flags=re.I)
        token = re.sub('[cgjkqsxz]+', '2', token, flags=re.I)
        token = re.sub('[dt]+', '3', token, flags=re.I)
        token = re.sub('[mn]+', '5', token, flags=re.I)
        token = re.sub('r+', '6', token, flags=re.I)
        token = token[k+1:]
        token = re.sub('[a@eio0uy]','', token, flags=re.I)
        soundex_result += token[0:3+k]
        if len(soundex_result) < 4+k:
            soundex_result += '0'*(4+k-len(soundex_result))
        return soundex_result

    def get_similars(self,
                    token,
                    level=0,
                    distance=2,
                    min_chars=3,
                    strict=True):

        def preprocess(token):
            return token

        if len(token) < min_chars:
            return []

        code = self.soundex(token, k=int(level))
        soundex_codes = self.level2code2token[str(level)]

        if code not in soundex_codes:
            return []

        cands = soundex_codes[code]
        cands = [self.idx2token[cand] for cand in cands]

        if strict:
            matchcode_token = matchcodex(token)
            cands = [cand for cand in cands if matchcodex(cand) == matchcode_token]

        dists = np.array([Levenshtein.distance(token.lower(), cand.lower()) for cand in cands])
        idx = np.where(dists <= distance)[0]

        cands = [list(cands)[i] for i in idx]
        cands = set([cand.lower() for cand in cands])

        rt = []
        for cand in cands:
            rt += [self.idx2token[a] for a in list(self.token2spell[self.token2idx[cand]])]

        rt = set([preprocess(a) for a in rt \
            if (good_pair_replacement(a[-1].lower(),token[-1].lower()) and len(a) > self.preserve_last_threshold) or
            (len(a) <= self.preserve_last_threshold and a[-1].lower() == token[-1].lower())])

        if strict:
            rt = set(a for a in rt if not final_check(token.lower(), a.lower()))

        return rt
anthro = ANTHRO()
anthro.load('/content/perturbations-in-the-wild/ANTHRO_Data_V1.0')


Cloning into 'perturbations-in-the-wild'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 47 (delta 18), reused 11 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 32.27 MiB | 6.02 MiB/s, done.
Resolving deltas: 100% (18/18), done.
Updating files: 100% (9/9), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 48.0 MB/s eta 0:00:00
loading... token2idx
loading... level2code2token
loading... token2spell
loaded
UPPD VERSION Public
Total unique codes:
Level 1: 407620 codes


100%|██████████| 1385430/1385430 [00:02<00:00, 643862.16it/s]


done


In [ ]:
!pip install textattack

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 39.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 42.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Relevant imports

In [ ]:
from textattack.transformations import WordSwapRandomCharacterDeletion
from textattack.transformations import WordSwapQWERTY
from textattack.transformations import CompositeTransformation

from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification

from textattack.augmentation import Augmenter

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def pert_using_textattack(word):
  transformation = CompositeTransformation(
      [WordSwapRandomCharacterDeletion(), WordSwapQWERTY()])
  # Set up constraints
  constraints = [RepeatModification(), StopwordModification()]
  # Create augmenter with specified parameters
  augmenter = Augmenter(
    transformation=transformation,
    constraints=constraints,
    pct_words_to_swap=0.1,
    transformations_per_example=10,
    )
  replacements = augmenter.augment(word)
  if replacements:
    return (random.choice(list(replacements)), True)
  else:
    return (word, False)

### Utillity functions for perturbations

In [ ]:
import random
import math
# Func to change an example
def pert_using_anthro(word):
  # Get possible replacements from Atnhro
  # Randomly choose one and return it
  replacements = anthro.get_similars(word, level=1, distance=5, strict=True)
  if replacements:
    return (random.choice(list(replacements)), True)
  else:
    return (word, False)


def perturb_focus_present(words_list, focus_words, pert_func, max_perturb_perc):
  # Iterate over words list, if a word is in focus words replace it
  # If replaced more than threshold - stop replacing
  tot_words = len(words_list)
  replaced_pairs = {}
  new_words = []
  for i, word in enumerate(words_list):
    if word in focus_words:
      new_word, replaced = pert_func(word)
      if replaced:
        replaced_pairs[word] = new_word
        new_words.append(new_word)
      else:
        new_words.append(word)
    else:
        new_words.append(word)

    if len(replaced_pairs)/tot_words > max_perturb_perc:
      break
  new_words += words_list[i+1:]
  return new_words, replaced_pairs

def perturb_no_focus(words_list, pert_func, max_perturb_perc):
  max_number = len(words_list)
  sample_size = math.ceil(max_perturb_perc * max_number)  # Number of integers to sample
  # Get a sample of integers from 0 to max_number-1
  sampled_integers = random.sample(range(max_number), sample_size)
  new_words = words_list.copy()
  replaced_pairs = {}
  for i in sampled_integers:
    new_word, replaced = pert_func(words_list[i])
    if replaced:
      new_words[i] = new_word
      replaced_pairs[words_list[i]] = new_word
  return new_words, replaced_pairs




In [ ]:
# If we want to change the perturbation mechanism we just need to replace the
# pert_func to be the new perturbation function.
def perturb(words_list, focus_words, pert_func, max_perturb_perc=0.1):
  words_set = set(words_list)
  focus_words_set = set(focus_words)
  if words_set.intersection(focus_words_set):
    return perturb_focus_present(words_list, focus_words, pert_func, max_perturb_perc)
  else:
    return perturb_no_focus(words_list, pert_func, max_perturb_perc)

### Perturbations sanity checks

In [ ]:
words = "I love persons and dogs".split(" ")
focus_words = ["persons", "dogs"]

replaced_words, replaced = perturb(words, focus_words, pert_using_textattack, 0.1)
print(replaced_words)
print(replaced)

['I', 'love', 'persobs', 'and', 'dogs']
{'persons': 'persobs'}


In [ ]:
words = "I love persons and dogs".split(" ")
focus_words = []

replaced_words, replaced = perturb(words, focus_words, pert_using_textattack, 0.1)
print(replaced_words)
print(replaced)

['I', 'love', 'persons', 'and', 'dos']
{'dogs': 'dos'}


### Perturb hate explain dataset

In [ ]:
# Iterate over original HateExplain dataset and add a replaced version for every entry
# Save to drive
!git clone https://github.com/alatteaday/mrp_hate-speech-detection.git
def load_dataset(dir, path):
  with open(os.path.join(dir, path), 'r') as f:
    dataset = json.load(f)
  return dataset
hate_explain_dir = "/content/mrp_hate-speech-detection/dataset"
file_name = 'hatexplain_two_div.json'
dataset = load_dataset(hate_explain_dir, file_name)["test"]

Cloning into 'mrp_hate-speech-detection'...
remote: Enumerating objects: 11526, done.
remote: Counting objects: 100% (11526/11526), done.
remote: Compressing objects: 100% (10519/10519), done.
remote: Total 11526 (delta 54), reused 11499 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (11526/11526), 6.53 MiB | 3.94 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Updating files: 100% (11484/11484), done.


List of 50 most common words in hateExplain for the hate classified text.
Compiled here:

In [ ]:
# focus_words = ['white',
#  'nigger',
#  'like',
#  'kike',
#  'retarded',
#  'jews',
#  'people',
#  'get',
#  'women',
#  'fucking',
#  'niggers',
#  'fuck',
#  'would',
#  'ghetto',
#  'faggot',
#  'muslim',
#  'shit',
#  'bitch',
#  'one',
#  'good',
#  'go',
#  'ass',
#  'hate',
#  'country',
#  'want',
#  'men',
#  'right',
#  'black',
#  'retard',
#  'back',
#  'trump',
#  'think',
#  'moslem',
#  'see',
#  'even',
#  'faggots',
#  'got',
#  'time',
#  'jew',
#  'gay',
#  'know',
#  'never',
#  'look',
#  'still',
#  'lol',
#  'need',
#  'could',
#  'america',
#  'well',
#  'take']

Using a list of hateful words to replace with ANTHRO

In [ ]:
# List of hateful words
hate_speech_words = [
    # Racial/Ethnic Slurs
    "nigger", "nigga", "mud shark", "chink", "gook", "spic", "wetback",
    "kike", "sand nigger", "towelhead", "beaner", "cracker", "honky",
    "jungle bunny", "redskin",

    # Gender-Based Slurs
    "bitch", "slut", "whore", "cunt", "feminazi", "dyke", "tranny",
    "she-male", "gold digger", "skank",

    # Sexual Orientation Slurs
    "faggot", "fag", "homo", "queer", "carpet muncher", "pillow biter",
    "fudge packer", "lesbo",

    # Religious Slurs
    "Christ killer", "jihadist", "infidel", "kike", "raghead", "heathen",
    "pagan", "Bible basher", "papist", "kaffir",

    # Disability-Related Slurs
    "retard", "cripple", "spaz", "mongoloid", "psycho", "nutjob",
    "lame", "vegetable", "deaf-mute", "imbecile",

    # Nationality/Cultural Slurs
    "yank", "brit", "kraut", "jap", "commie", "ruski", "paki",
    "wop", "dago", "polack",

    # Appearance-Based Slurs
    "fatty", "lardass", "twig", "beanpole", "ugly", "four-eyes",
    "baldie", "geek", "nerd", "pimple-face"
]


In [ ]:
# Replace the hateful words inside the list of hateful speech
all_replaced_pairs = {}
tot_words = 0
for i, row in enumerate(tqdm(dataset)):
  words_list = row["text"]
  # focus_words = row["target1"] + row["target2"] + row["target3"]
  # focus_words = [w.lower() for w in focus_words]
  replaced_words, replaced_pairs = perturb(words_list, hate_speech_words, pert_using_anthro, 0.2)
  row["pert_list_text"] = replaced_words
  row["replaced_pairs"] = replaced_pairs
  all_replaced_pairs.update(replaced_pairs)
  tot_words += len(words_list)
  if i < 5:
    print(row)
    # print(hate_speech_words)
    print(words_list)
    print(replaced_words)
    print(replaced_pairs)
  if i % 5 == 0:
    # print(hate_speech_words)
    print(words_list)
    print(replaced_words)
    print(replaced_pairs)

print(len(all_replaced_pairs))
print(tot_words)
print(len(all_replaced_pairs)/tot_words)

  0%|          | 1/1924 [00:28<15:05:09, 28.24s/it]

{'post_id': '1178610029273976833_twitter', 'text': ['<user>', 'men', 'can', 'not', 'be', 'raped', 'can', 'not', 'be', 'abused', 'that', 'why', 'they', 'call', 'it', 'violence', 'against', 'women', 'and', 'children', 'because', 'men', 'are', 'always', 'the', 'perpetrators', 'women', 'come', 'first', 'even', 'before', 'children'], 'annotatorid1': 9, 'target1': ['None'], 'label1': 'normal', 'annotatorid2': 17, 'target2': ['None'], 'label2': 'normal', 'annotatorid3': 64, 'target3': ['None'], 'label3': 'normal', 'rationales': [], 'final_label': 'non-toxic', 'final_target_category': 'None', 'pert_list_text': ['<user>', 'men', 'caN', 'noooott', 'be', 'raped', 'can', 'not', 'be', 'aBuSeD', 'that', 'why', 'they', 'call', 'it', 'violence', 'against', 'women', 'ANd', 'childreen', 'because', 'men', 'are', 'always', 'the', 'perpetrtors', 'women', 'come', 'FIRST', 'even', 'before', 'children'], 'replaced_pairs': {'and': 'ANd', 'children': 'childreen', 'perpetrators': 'perpetrtors', 'abused': 'aBuSeD

  0%|          | 2/1924 [00:37<8:59:36, 16.85s/it] 

{'post_id': '1165785686903009283_twitter', 'text': ['<user>', 'you', 'are', 'missing', 'an', 'essential', 'premise', 'whether', 'any', 'of', 'those', 'immigrants', 'committed', 'any', 'crimes', 'dumbass'], 'annotatorid1': 200, 'target1': ['Refugee'], 'label1': 'offensive', 'annotatorid2': 199, 'target2': ['Refugee'], 'label2': 'normal', 'annotatorid3': 211, 'target3': ['None'], 'label3': 'normal', 'rationales': [], 'final_label': 'non-toxic', 'final_target_category': ['Refugee'], 'pert_list_text': ['<user>', 'you', 'are', 'missing', 'an', 'essentiall', 'premise', 'whether', 'Any', 'of', 'those', 'immigrants', 'comitteed', 'any', 'crimes', 'dumbass'], 'replaced_pairs': {'essential': 'essentiall', 'committed': 'comitteed', 'any': 'Any'}}
['nigger', 'nigga', 'mud shark', 'chink', 'gook', 'spic', 'wetback', 'kike', 'sand nigger', 'towelhead', 'beaner', 'cracker', 'honky', 'jungle bunny', 'redskin', 'bitch', 'slut', 'whore', 'cunt', 'feminazi', 'dyke', 'tranny', 'she-male', 'gold digger', '

  0%|          | 3/1924 [01:07<12:18:03, 23.05s/it]

...

['nigger', 'nigga', 'mud shark', 'chink', 'gook', 'spic', 'wetback', 'kike', 'sand nigger', 'towelhead', 'beaner', 'cracker', 'honky', 'jungle bunny', 'redskin', 'bitch', 'slut', 'whore', 'cunt', 'feminazi', 'dyke', 'tranny', 'she-male', 'gold digger', 'skank', 'faggot', 'fag', 'homo', 'queer', 'carpet muncher', 'pillow biter', 'fudge packer', 'lesbo', 'Christ killer', 'jihadist', 'infidel', 'kike', 'raghead', 'heathen', 'pagan', 'Bible basher', 'papist', 'kaffir', 'retard', 'cripple', 'spaz', 'mongoloid', 'psycho', 'nutjob', 'lame', 'vegetable', 'deaf-mute', 'imbecile', 'yank', 'brit', 'kraut', 'jap', 'commie', 'ruski', 'paki', 'wop', 'dago', 'polack', 'fatty', 'lardass', 'twig', 'beanpole', 'ugly', 'four-eyes', 'baldie', 'geek', 'nerd', 'pimple-face']
['objectively', 'east', 'asians', 'are', 'an', 'upgrade', 'intelligence', 'and', 'the', 'mixes', 'are', 'frequently', 'stunning', 'look', 'at', 'kristin', 'kreuk', 'dutch', 'chinese', 'and', 'hybrid', 'vigor', 'is', 'a', 'thing']
['obje

100%|█████████▉| 1921/1924 [6:28:31<00:34, 11.51s/it]

['nigger', 'nigga', 'mud shark', 'chink', 'gook', 'spic', 'wetback', 'kike', 'sand nigger', 'towelhead', 'beaner', 'cracker', 'honky', 'jungle bunny', 'redskin', 'bitch', 'slut', 'whore', 'cunt', 'feminazi', 'dyke', 'tranny', 'she-male', 'gold digger', 'skank', 'faggot', 'fag', 'homo', 'queer', 'carpet muncher', 'pillow biter', 'fudge packer', 'lesbo', 'Christ killer', 'jihadist', 'infidel', 'kike', 'raghead', 'heathen', 'pagan', 'Bible basher', 'papist', 'kaffir', 'retard', 'cripple', 'spaz', 'mongoloid', 'psycho', 'nutjob', 'lame', 'vegetable', 'deaf-mute', 'imbecile', 'yank', 'brit', 'kraut', 'jap', 'commie', 'ruski', 'paki', 'wop', 'dago', 'polack', 'fatty', 'lardass', 'twig', 'beanpole', 'ugly', 'four-eyes', 'baldie', 'geek', 'nerd', 'pimple-face']
['he', 'sounds', 'like', 'a', 'clothing', 'line', 'i', 'will', 'start', 'a', 'line', 'of', 'robes', 'called', 'tommy', 'goatfucker']
['he', 'sounds', 'like', 'a', 'clothing', 'LInE', 'i', 'will', 'start', 'a', 'line', 'of', 'robes', 'Ca

100%|██████████| 1924/1924 [6:29:08<00:00, 12.14s/it]

2101
44519
0.047193333183584533


In [ ]:
from google.colab import drive
import json
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Path to save the file in Google Drive
file_path = '/content/drive/My Drive/NLP/perturbed_list_GPT_hate_explain.json'

# Save the dictionary as a JSON file
with open(file_path, 'w') as json_file:
    json.dump(dataset, json_file)

print(f"File saved to: {file_path}")


File saved to: /content/drive/My Drive/NLP/perturbed_list_GPT_hate_explain.json
